## Part 1. Traversing a Web Graph (8 points)

You begin by building a web crawler function, webCrawler, that traverses a web graph consisting of a self-contained set of linked web pages.

First, you can use the `urllib` package to retrieve web pages as follows:

```

import urllib.request
webUrl  = urllib.request.urlopen('https://ischool.berkeley.edu/')
data = webUrl.read()

```

Starting with the following URL:

[https://people.ischool.berkeley.edu/~chuang/i206/b5/index.html](https://people.ischool.berkeley.edu/~chuang/i206/b5/index.html)

Your crawler should identify and follow the links on the page, as well as the links found on the other pages reachable from this source page, using the breadth-first search (BFS) technique. 

You can use regex (which you have now mastered) or the [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/). library (e.g., its `findAll()` and `get()` methods) to find the links on each page. To simplify your task, all the links in this set of web pages use relative links, e.g., 

\<a href=”somepage.html”>This is a link to some random page</a>

which should be resolved to:

[https://people.ischool.berkeley.edu/~chuang/i206/b5/somepage.html](https://people.ischool.berkeley.edu/~chuang/i206/b5/somepage.html)

Note that pages may link to one another via loops, e.g., A links to B, B links to C, and C links back to A. Your crawler has to avoid loops by keeping track of which pages have already been visited (or not), so that you don't visit the same pages again. Use Python's `deque` data structure to implement a queue for this purpose.

Note: please do not try to run your code on the open Web unless you have properly implemented the following: (i) checking and conforming to a site’s robots.txt file, (ii) rate-limiting your crawler, (iii) properly resolving fully specified and relative links. Otherwise you may get a nasty call from someone.

Upon completion of the crawl, your crawler function should return the following:

A list of the pages found (following the exact order in which they were visited, starting with `index.html`)
Total number of pages crawled (including `index.html`)
Total number of links found
 